In [1]:
import envpool
import numpy as np

In [14]:
env = envpool.make_gymnasium('Asterix-v5', num_envs=8, full_action_space=True, reward_clip=True, episodic_life=True)

In [15]:
env.reset();

In [16]:
for _ in range(100):
    obs, rew, term, trunc, info = env.step(np.random.randint(0, 18, 8))
    # print(rew, info['reward'])
    print(term, info['terminated'], info['lives'])

[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3]
[False False False False False False False False] [0 0

In [17]:
from my_agents import *

In [18]:
agent = StackedCNNAgent(18, 4)

In [19]:
sum([p.numel() for p in agent.parameters()])

1693875

In [29]:
class Agent(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            layer_init(nn.Conv2d(4, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(64 * 7 * 7, 512)),
            nn.ReLU(),
        )
        self.actor = layer_init(nn.Linear(512, 6), std=0.01)
        self.critic = layer_init(nn.Linear(512, 1), std=1)

    def get_value(self, x):
        return self.critic(self.network(x / 255.0))

    def get_action_and_value(self, x, action=None):
        hidden = self.network(x / 255.0)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)

In [30]:
agent = Agent()
sum([p.numel() for p in agent.parameters()])

1687719